<a href="https://colab.research.google.com/github/SamOryeJack/AI/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install flask flask-ngrok transformers torch pyngrok

In [7]:
import os
from huggingface_hub import login
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok

# Set your Hugging Face token
os.environ['HUGGINGFACE_TOKEN'] = 'hf_key'

# Authenticate with Hugging Face using the token
login(token=os.getenv('HUGGINGFACE_TOKEN'))

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set the pad token
tokenizer.pad_token = tokenizer.eos_token

# Define a function to generate text
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True)
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=150,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.7,        # Increase randomness
        top_p=0.9,              # Use nucleus sampling
        repetition_penalty=1.2  # Penalize repetition
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Set your ngrok authtoken
ngrok.set_auth_token("ngrok_key")

# Create an ngrok tunnel to expose the local server
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# Initialize Flask app
app = Flask(__name__)

# Define the HTML template
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chatbot</title>
</head>
<body>
    <h1>Chatbot Interface</h1>
    <form action="/chatbot" method="post" id="chat-form">
        <label for="prompt">Enter your prompt:</label><br><br>
        <input type="text" id="prompt" name="prompt" style="width: 300px;"><br><br>
        <input type="submit" value="Submit">
    </form>
    <div id="response">
        {% if response %}
            <h2>Response:</h2>
            <p>{{ response }}</p>
        {% endif %}
    </div>
    <script>
        document.getElementById('chat-form').onsubmit = async function(event) {
            event.preventDefault();
            const prompt = document.getElementById('prompt').value;
            const responseDiv = document.getElementById('response');
            const response = await fetch('/chatbot', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({input: prompt}),
            }).then(res => res.json());
            responseDiv.innerHTML = '<h2>Response:</h2><p>' + response.response + '</p>';
        }
    </script>
</body>
</html>
"""

# Define a route for the chatbot interface
@app.route('/', methods=['GET', 'POST'])
def home():
    return render_template_string(html_template)

# Define a route for the chatbot API
@app.route('/chatbot', methods=['POST'])
def chatbot():
    data = request.get_json()
    user_input = data['input']
    response = generate_text(user_input)
    return jsonify({'response': response})

# Start the Flask application
if __name__ == '__main__':
    app.run(port=5000, host='0.0.0.0')


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful
 * ngrok tunnel "NgrokTunnel: "https://d242-35-229-43-75.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 04:28:28] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 04:28:28] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 04:28:44] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 04:29:16] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 04:29:48] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 04:29:55] "POST /chatbot HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 04:31:03] "POST /chatbot HTTP/1.1" 200 -
